<a href="https://colab.research.google.com/github/jade0520/DeepLearning_FromScratch/blob/master/CH05/Ch05_Backpropergation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

오차 역전파법

In [ ]:
!git clone https://github.com/WegraLee/deep-learning-from-scratch.git

Cloning into 'deep-learning-from-scratch'...
remote: Enumerating objects: 791, done.
remote: Total 791 (delta 0), reused 0 (delta 0), pack-reused 791
Receiving objects: 100% (791/791), 52.15 MiB | 43.38 MiB/s, done.
Resolving deltas: 100% (457/457), done.


In [ ]:
import numpy as np

#단순한 계층 구현하기


> 노드의 종류마다 순전파/역전파를 묶은 클래스를 생성한다.



곱셈을 위한 계층 구현 

In [ ]:
class MulLayer:
      #초기화 
      def __init__(self):
        self.x = None
        self.y = None

      #순방향 : 단순 곱셈
      def forward(self,x,y):
        self.x = x
        self.y = y
        out = x*y

        return out

      #역방향 : 앞의 값 * 반대쪽 값
      def backward(self,dout):
        dx = dout * self.y                  #전하기만 하면 되므로 따로 클래스의 메서드로 만들어 줄 필요 x
        dy = dout * self.x

        return dx,dy

곱셈의 순전파

In [ ]:
apple = 100
apple_num = 2
tax = 1.1

#계층들 : 구현해 놓은 MulLayer계층
mul_apple_layer = MulLayer()  #첫번째 사과 개수 * 가격
mul_tax_layer = MulLayer()    #두번째 세금 * 사과 개수 * 가격

#순전파
apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(price)

220.00000000000003


곱셈의 역전파

In [ ]:
dprice = 1
dapple_price,dtax = mul_tax_layer.backward(dprice)
dapple , dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num,dtax)



2.2 110.00000000000001 200


덧셈 계층 구현

In [ ]:
class AddLayer:
      # add 의 역전파는 앞의 결과를 그대로 흘려보내기 때문에
      # 따로 덧셈의 순전파 결과를 기억해 두지 않아도 된다.
      def __init__(self): 
          pass
      
      def forward(self,x,y):
        out = x+y
      
        return out


      def backward(self,dout):
        dx = dout
        dy = dout
      
        return dx, dy

In [ ]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

#역전파 
dprice = 1
dall_price,dtax = mul_tax_layer.backward(dprice)
dapple_price , dorange_price = add_apple_orange_layer.backward(dall_price)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)
dorange,dorange_num = mul_orange_layer.backward(dorange_price)

print(price)
print(dapple_num,dapple,dorange,dorange_num,dtax)


715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


#활성화 함수 계층 구현하기

[ReLU 계층]

In [ ]:
class Relu:
      def __init__(self):
        self.mask =none
      
      def forward(self,x):
        #입력 값들 중 0 보다 작은 애들 따로 표시 해두는 것.
        self.mask = (x<=0)
        out = x.copy()

        # 0보다 작았던 입력값들은 모두 0으로 셋팅한다.
        out[self.mask] = 0

        return out

      def backward(self, dout):
         # 입력값이 0보다 작았던 경우, 역전파 결과도 모두 0으로 셋팅한다.
        dout[self.mask] = 0
        dx = dout

        return dx

In [ ]:
x =  np.array([[1.0, -1.5], [-2.0, 3.0]])
print(x)

#작은 경우만 true가 됨을 확인.
mask = (x<=0)
print(mask)

[[ 1.  -1.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


[Sigmoid 계층]

In [ ]:
class Sigmoid:
      #순전파 시 출력 보관 했다가 역전파 시에 사용
      def __init__(self):
        self.out = None

      def forward(self,x):
        out = 1/(1+np.exp(-x))
        self.out = out

        return out

      def backward(self,dout):
        dx = dout * (1-self.out)* self.out
     
        return dx


#Affine/Softmax 계층구현  

  Affine 계층

In [ ]:
 class Affine:
      def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dw = None
        delf.db = None

      def forward(self,x):
        self.x = x
        
        #AFfine : just dot product
        out = np.dot(x,self.W) +self.b  #브로드 캐스트

        return out

      def backward(self, dout):
        dx =np.dot(dout,self.W.T)
        self.dW =np.dot(self.x.T,dout)
       
        # 역전파 시 데이터의 각 역전파 값이 편향의 원소에 모여야 한다.  
        self.db = np.sum(dout,axis =0)

        return dx

soft-with Loss

In [ ]:
class SoftWithLoss:
      def __init__(self):
        self.loss = None #손실
        self.y = None
        self.t = None

      #softmax + loss
      def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        return self.loss

      def backward(self,dout = 1):
        # dout 시작 부분이므로 1
        batch_size = self.t.shape[0]
        #결국에 신경망의 출력과 정답레이블의 오차를 앞으로 전달하게 된다.
        #데이터 1개당 오차를 전파함
        dx = (self.y - self.t) / batch_size

        return dx

#오차 역전파법 구현

In [ ]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
os.chdir('/content/deep-learning-from-scratch/')

from common.layers import * 
from collections import OrderedDict



class TwoLayerNet:
  def __init__(self,input_size, hidden_size, output_size, weight_init_std = 0.01):

    # initialize weights
    self.params = {}  #dictionary
    self.params['W1'] = weight_init_std *np.random.randn(input_size,hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
    self.params['b2'] = np.zeros(output_size)

    #layers : Oder Dictionary!
    self.layers  = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'],self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'],self.params['b2'])

    self.lastLayer = SoftmaxWithLoss()


  def predict(self,x):
    #순전파로 예측
      for layer in self.layers.values():
          x = layer.forward(x)   #do function forward ,given layer
      
      return x
  
  #학습 시 이용
  def loss(self,x,t):
    y = self.predict(x) # 순전파로 얻은 값
    return self.lastLayer.forward(y,t) #출력층 통해서 값 리턴

  #추론 시 이용
  def accuracy(self,x,t):
      y = self.predict(x)
      y = np.argmax(y,axis = 1)

      #if t is not number label, if it is one-hot label
      #받은 t의 최대 값만  t에 저장
      if t.ndim != 1 : t = np.argmax(t,axis = 1)

      #compare predicted y and answer t
      accuracy = np.sum(y==t) / float(x.shape[0]) 

      return accuracy

  def numerical_gradient(self, x, t):
      loss_W = lambda W: self.loss(x, t)
        
      grads = {}
      grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
      grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
      grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
      grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
      return grads
        
  def gradient(self, x, t):
        # forward
      self.loss(x, t)

        # backward
      dout = 1
      dout = self.lastLayer.backward(dout)
        
      layers = list(self.layers.values())
      layers.reverse()
      for layer in layers:
          dout = layer.backward(dout)

        # 결과 저장
      grads = {}
      grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
      grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

      return grads


In [ ]:
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)


0.14605 0.1567
0.8998666666666667 0.9026
0.9196833333333333 0.9227
0.9345666666666667 0.9347
0.9429 0.9416
0.9473 0.9455
0.95395 0.949
0.9597333333333333 0.9557
0.9618833333333333 0.9588
0.9659333333333333 0.9596
0.9659166666666666 0.961
0.9693166666666667 0.9627
0.9716666666666667 0.9633
0.9737666666666667 0.9652
0.9743833333333334 0.9669
0.9772333333333333 0.9686
0.9774 0.9686
